In [2]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [4]:
base_data_dir = '/srv/scratch/z5370003/projects/data/groundwater/FEFLOW/coastal/variable_density/'

forcings_dir_name = 'forcings'
filtered_dir_name = 'filter_all_ts'
raw_dir_name = 'all'

# forcings_out_dir_name = 'forcings_corrected'
forcings_out_dir_name = 'forcings_corrected_all'

# data_dir_name = filtered_dir_name
data_dir_name = raw_dir_name


In [6]:
data_path = os.path.join(base_data_dir, data_dir_name)

forcings_dir_path = os.path.join(base_data_dir, forcings_dir_name)
forcings_out_dir_path = os.path.join(base_data_dir, forcings_out_dir_name)
os.makedirs(forcings_out_dir_path, exist_ok=True)

In [7]:
for ts_file in tqdm(os.listdir(data_path)):
    
    data_at_ts_path = os.path.join(data_path, ts_file)
    forcings_data_at_ts_path = os.path.join(forcings_dir_path, ts_file)

    drop_data_columns = ['head', 'pressure', 'mass_concentration']
    data_df = pd.read_csv(data_at_ts_path).drop(columns=drop_data_columns)

    forcings_cols = ['node', 'mass_concentration_bc', 'hydraulic_head_bc', 'recharge_forcing', 'sea_level_forcing']
    forcings_df = pd.read_csv(forcings_data_at_ts_path).loc[:, forcings_cols]
    forcings_df.rename(columns={'hydraulic_head_bc': 'head_bc'}, inplace=True)

    merged_df = pd.merge(data_df, forcings_df, on='node', how='left')
    
    merged_df.to_csv(os.path.join(forcings_out_dir_path, ts_file), index=False)
    

100%|██████████| 1909/1909 [16:05<00:00,  1.98it/s]


In [8]:
# Create color array: use head_bc where available, otherwise use a special value for grey
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'iframe'

colors = merged_df['sea_level_forcing']#.fillna(-999)  # Use -999 as placeholder for NaN

fig = go.Figure(data=[go.Scatter3d(
    x=merged_df['X'],
    y=merged_df['Y'],
    z=merged_df['Z'],
    mode='markers',
    marker=dict(
        size=5,
        color=colors,
        # colorscale=[[0, 'grey'], [0.001, '#440154'], [0.1111111111111111, '#482878'], 
        #             [0.2222222222222222, '#3e4989'], [0.3333333333333333, '#31688e'], 
        #             [0.4444444444444444, '#26828e'], [0.5555555555555556, '#1f9e89'], 
        #             [0.6666666666666666, '#35b779'], [0.7777777777777778, '#6ece58'], 
        #             [0.8888888888888888, '#b5de2b'], [1.0, '#fde725']],
        cmin=merged_df['sea_level_forcing'].min(),
        cmax=merged_df['sea_level_forcing'].max(),
        showscale=True,
        colorbar=dict(title="sea_level_forcing")
    ),
    text=merged_df['sea_level_forcing'].fillna('No BC'),
    hovertemplate='X: %{x}<br>Y: %{y}<br>Z: %{z}<br>Head BC: %{text}<extra></extra>'
)])

fig.update_layout(
    title='sea_level_forcing - 3D Scatter Plot',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

fig.show()

In [7]:
colors = merged_df['recharge_forcing']#.fillna(-999)  # Use -999 as placeholder for NaN

fig = go.Figure(data=[go.Scatter3d(
    x=merged_df['X'],
    y=merged_df['Y'],
    z=merged_df['Z'],
    mode='markers',
    marker=dict(
        size=5,
        color=colors,
        # colorscale=[[0, 'grey'], [0.001, '#440154'], [0.1111111111111111, '#482878'], 
                    # [0.2222222222222222, '#3e4989'], [0.3333333333333333, '#31688e'], 
                    # [0.4444444444444444, '#26828e'], [0.5555555555555556, '#1f9e89'], 
                    # [0.6666666666666666, '#35b779'], [0.7777777777777778, '#6ece58'], 
                    # [0.8888888888888888, '#b5de2b'], [1.0, '#fde725']],
        cmin=merged_df['recharge_forcing'].min(),
        cmax=merged_df['recharge_forcing'].max(),
        showscale=True,
        colorbar=dict(title="recharge_forcing")
    ),
    text=merged_df['recharge_forcing'].fillna('NA'),
    hovertemplate='X: %{x}<br>Y: %{y}<br>Z: %{z}<br>Recharge: %{text}<extra></extra>'
)])

fig.update_layout(
    title='Recharge - 3D Scatter Plot',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

fig.show()